
<p>Prerequisites: install <a href="https://www.tensorflow.org/install/">tensorflow 1.0</a> and  <a href="http://scikit-image.org/docs/dev/api/skimage.html">scikit-image</a>.</p>
<p>clone this fork of <a href="https://github.com/marcotcr/tf-models">tf-slim</a> somewhere
download <a href="http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz">the pretrained model</a> and put it in tf-models/slim/pretrained/</p>


In [12]:
pip install tensorflow== 1.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow== (from versions: 1.13.0rc1, 1.13.0rc2, 1.13.1, 1.13.2, 1.14.0rc0, 1.14.0rc1, 1.14.0, 1.15.0rc0, 1.15.0rc1, 1.15.0rc2, 1.15.0rc3, 1.15.0, 1.15.2, 1.15.3, 2.0.0a0, 2.0.0b0, 2.0.0b1, 2.0.0rc0, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.1.0rc0, 2.1.0rc1, 2.1.0rc2, 2.1.0, 2.1.1, 2.2.0rc0, 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0)
ERROR: No matching distribution found for tensorflow==


In [17]:
pip install tf_slim

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
slim = tf.contrib.slim

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
print(tf.__version__)

1.15.0


In [3]:
import sys
sys.path.append('/Users/marcotcr/phd/tf-models/slim')
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


<h2 id="Create-a-predict-fn-for-inception-v3,-takes-in-a-list-of-images-and-returns-a-matrix-of-prediction-probabilities">Create a predict fn for inception v3, takes in a list of images and returns a matrix of prediction probabilities<a class="anchor-link" href="https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20images.html#Create-a-predict-fn-for-inception-v3,-takes-in-a-list-of-images-and-returns-a-matrix-of-prediction-probabilities">¶</a></h2>


In [4]:
import sys
sys.path.append('C:/Users/wangu/models/research/slim/')

In [7]:
import tf_slim
from nets import inception_resnet_v2

In [8]:
from nets import inception
from preprocessing import inception_preprocessing

In [15]:
session = tf.Session()

In [16]:

image_size = inception.inception_v3.default_image_size
def transform_img_fn(path_list):
    out = []
    for f in path_list:
        image_raw = tf.image.decode_jpeg(open(f).read(), channels=3)
        image = inception_preprocessing.preprocess_image(image_raw, image_size, image_size, is_training=False)
        out.append(image)
    return session.run([out])[0]



In [17]:
image_size

299

In [31]:

from datasets import imagenet
names = imagenet.create_readable_names_for_imagenet_labels()



HTTPError: HTTP Error 404: Not Found

In [18]:

processed_images = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))


In [23]:

import os
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False, reuse=True)
probabilities = tf.nn.softmax(logits)

checkpoints_dir = '/Users/marcotcr/phd/tf-models/slim/pretrained'
init_fn = slim.assign_from_checkpoint_fn(
    os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
    slim.get_model_variables('InceptionV3'))
init_fn(session)

NotFoundError: FindFirstFile failed for: /Users/marcotcr/phd/tf-models/slim/pretrained : The system cannot find the path specified.
; No such process

In [ ]:

def predict_fn(images):
    return session.run(probabilities, feed_dict={processed_images: images})




<h2 id="Let's-see-the-top-5-prediction-for-some-image">Let's see the top 5 prediction for some image<a class="anchor-link" href="https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20images.html#Let's-see-the-top-5-prediction-for-some-image">¶</a></h2>


In [ ]:

images = transform_img_fn(['dogs.jpg'])
# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(images[0] / 2 + 0.5)
preds = predict_fn(images)
for x in preds.argsort()[0][-5:]:
    print x, names[x], preds[0,x]



In [ ]:

image = images[0]



In [ ]:

## Now let's get an explanation



In [ ]:

from lime import lime_image
import time



In [ ]:

explainer = lime_image.LimeImageExplainer()




<p>hide_color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels. Here, we set it to 0 (in the representation used by inception model, 0 means gray)</p>


In [ ]:

tmp = time.time()
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
explanation = explainer.explain_instance(image, predict_fn, top_labels=5, hide_color=0, num_samples=1000)
print time.time() - tmp




<p>Image classifiers are a bit slow. Notice that an explanation in my macbookpro took 7.65 minutes</p>



<h3 id="Now-let's-see-the-explanation-for-the-top-class-(Bernese-mountain-dog)">Now let's see the explanation for the top class (Bernese mountain dog)<a class="anchor-link" href="https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20images.html#Now-let's-see-the-explanation-for-the-top-class-(Bernese-mountain-dog)">¶</a></h3>



<p>We can see the top 5 superpixels that are most positive towards the class with the rest of the image hidden</p>


In [ ]:

from skimage.segmentation import mark_boundaries



In [ ]:

temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))




<p>Or with the rest of the image present:</p>


In [ ]:

temp, mask = explanation.get_image_and_mask(240, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))




<p>We can also see the 'pros and cons' (pros in green, cons in red)</p>


In [ ]:

temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))




<p>Or the pros and cons that have weight at least 0.1</p>


In [ ]:

temp, mask = explanation.get_image_and_mask(240, positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))




<h3 id="Let's-see-the-explanation-for-Egyptian-cat">Let's see the explanation for Egyptian cat<a class="anchor-link" href="https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20images.html#Let's-see-the-explanation-for-Egyptian-cat">¶</a></h3>



<p>Most positive towards egyptian cat:</p>


In [ ]:

temp, mask = explanation.get_image_and_mask(286, positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))




<p>Pros and cons:</p>


In [ ]:

temp, mask = explanation.get_image_and_mask(286, positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

